In [1]:
# change working directory
%cd ..

d:\Projects\Github\Dit Diffusion


In [2]:
# import torch
# from transformers import CLIPTokenizer, CLIPTextModel

# # Set device
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Load tokenizer and text encoder
# tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
# text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device).eval()

# # Example captions
# captions = [
#     "a photo of a smiling woman",
#     "a man wearing sunglasses",
#     "a cat sitting on a chair",
#     "a painting of a futuristic city"
# ]

# # Tokenize
# inputs = tokenizer(captions, padding="max_length", truncation=True, return_tensors="pt").to(device)

# # Get embeddings
# with torch.no_grad():
#     outputs = text_encoder(**inputs)
#     embeddings = outputs.last_hidden_state  # shape: (B, 77, 768)
#     pooled = outputs.pooler_output          # shape: (B, 768), can be used directly

# print("Token Embeddings shape:", embeddings.shape)
# print("Pooled Embeddings shape:", pooled.shape)


In [3]:
# embeddings

In [4]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm

from diffusers import AutoencoderKL, DDPMScheduler, DiTTransformer2DModel

from utils.ema import create_ema_model
from utils.checkpoint import save_training_state, load_training_state
from utils.celeba_with_caption import CelebAloader
from utils.metrics.gpu import init_nvml, gpu_info
from omegaconf import OmegaConf
import lpips

c:\Users\Incognito-R\miniconda3\envs\ml_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.cuda.empty_cache()

In [6]:
torch.manual_seed(0)
handle = init_nvml()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Enable mixed precision training
scaler = torch.amp.GradScaler('cuda') if device == "cuda" else None
print("Mixed precision training enabled" if scaler is not None else "Mixed precision training disabled")

# Load configuration
config = OmegaConf.load("configs/train_config_256.yaml")
# config = OmegaConf.load("configs/train_config_512.yaml"
print(f"Configuration loaded: {OmegaConf.to_yaml(config)}")

Using device: cuda
Mixed precision training enabled
Configuration loaded: data:
  path: data/CelebA-HQ/images_512
  parquet_path: /dbfs/mnt/ds-space/Hitesh/Datasets/CelebA-HQ/parquet_files/CelebA-HQ.parquet
  image_size: 256
  normalize: true
  caption_path: data/CelebA-HQ/captions.csv
checkpoint:
  path: Checkpoints/
  ckpt_name: dit_diffusion_ckpt_256.pth
  ema_ckpt_name: dit_diffusion_ema_ckpt_256.pth
output_dir:
  train: output/train
  test: output/test
training:
  batch_size: 12
  validation_split: 0
  epochs: 100
  warmup_epochs: 10
  lr: 0.0001
  grad_accum_steps: 2
  use_ema: true
  ema_beta: 0.995
  step_start_ema: 2000
  num_workers: 4
sampling:
  dir: output/samples
  num_samples: 25
  steps: 50
model:
  type: dit
  latent_dim: 4
  img_size: 32
  hidden_size: 768
  depth: 12
  num_heads: 12
  attn_head_dim: 64
  patch_size: 1
scheduler:
  type: squaredcos_cap_v2
  timesteps: 1000
  beta_start: 0.0001
  beta_end: 0.02
logs: logs



In [7]:
# === Load VAE from diffusers ===
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema").to(device).eval()

# === Load DiT from diffusers ===

model = DiTTransformer2DModel(
    in_channels=config.model.latent_dim,
    num_attention_heads=config.model.num_heads,
    attention_head_dim=config.model.attn_head_dim,
    num_layers=config.model.depth,
    sample_size=config.model.img_size // config.model.patch_size,
    patch_size=config.model.patch_size,
).to(device)

# === Load noise scheduler from diffusers ===
scheduler = DDPMScheduler(
    num_train_timesteps=config.scheduler.timesteps,
    beta_start=config.scheduler.beta_start,
    beta_end=config.scheduler.beta_end,
    beta_schedule="linear",
)

# EMA model
ema_model, ema = create_ema_model(model, beta=config.training.ema_beta, step_start_ema=config.training.step_start_ema)

# Optimizer for DiT model
optimizer = torch.optim.AdamW(model.parameters(), lr=config.training.lr)

# losses
MSE_LOSS_Dit = torch.nn.MSELoss()
LPIPS_LOSS = lpips.LPIPS(net='vgg').to(device).eval()

print("Models, optimizers, losses initialized successfully.")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


c:\Users\Incognito-R\miniconda3\envs\ml_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Incognito-R\miniconda3\envs\ml_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\Incognito-R\miniconda3\envs\ml_env\Lib\site-packages\lpips\weights\v0.1\vgg.pth
Models, optimizers, losses initialized successfully.


In [8]:
# === Load data ===
dataloader, _ = CelebAloader(data_config=config.data, train_config=config.training)

print(f"Dataset size: {len(dataloader.dataset)} images")
# batch = next(iter(dataloader))

Dataset size: 30000 images


In [ ]:
dataloader['image']

In [ ]:
for batch in dataloader:
    print('helloe')
    break

In [ ]:
print(f"Batch shape: {batch.shape}, Device: {batch.device}")

In [1]:
cd ..

w:\Projects\Diffusion Model\UNet-Diffusion


In [6]:
detector.__class__.__annotations__

{}